# 项目：整理Netflix电影演员评分数据

## 分析目标

此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

本实战项目的目的在于练习整理数据，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netflix电视剧及电影数据。数据集包含两个数据表：`titles.csv`和`credits.csv`。

`titles.csv`包含电影及电视剧相关信息，包括影视作品ID、标题、类型、描述、流派、IMDB（一个国外的在线评分网站）评分，等等。`credits.csv`包含超过7万名出现在Netflix影视作品的导演及演员信息，包括名字、影视作品ID、人物名、演职员类型（导演/演员）等。

`titles.csv`每列的含义如下：
- id：影视作品ID。
- title：影视作品标题。
- show_type：作品类型，电视节目或电影。
- description：简短描述。
- release_year：发布年份。
- age_certification：适龄认证。
- runtime：每集电视剧或电影的长度。
- genres：流派类型列表。
- production_countries：出品国家列表。
- seasons：如果是电视剧，则是季数。
- imdb_id：IMDB的ID。
- imdb_score：IMDB的评分。
- imdb_votes：IMDB的投票数。
- tmdb_popularity：TMDB的流行度。
- tmdb_score：TMDB的评分。

`credits.csv`每列的含义如下：
- person_ID：演职员ID。
- id：参与的影视作品ID。
- name：姓名。
- character_name：角色姓名。
- role：演职员类型，演员或导演。

In [1]:
import pandas as pd

**读取数据**

In [2]:
original_titles = pd.read_csv("titles.csv")
original_credits = pd.read_csv("credits.csv")

**评估**

In [3]:
original_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [4]:
original_credits.head()

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


**结构上：每个变量一列，每个观察值一行，每个观察单位一格**  
**内容上：是否丢失数据，是否重复数据，是否有无效数据**

先搞个副本，别把原本污染了

In [5]:
cleaned_titles = original_titles.copy()

In [6]:
cleaned_credits = original_credits.copy()

先看一遍有什么问题

In [7]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5850 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5850 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5850 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5850 non-null   int64  
 7   genres                5850 non-null   object 
 8   production_countries  5850 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5447 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
dtypes: float64(5), int64(

我们看title缺失了一行，我们把它单独提取出来看看

In [8]:
print(cleaned_titles[cleaned_titles['title'].isna()])

             id title   type description  release_year age_certification  \
1704  tm1063792   NaN  MOVIE         NaN          2015               NaN   

      runtime genres production_countries  seasons    imdb_id  imdb_score  \
1704       11     []                   []      NaN  tt4661188         NaN   

      imdb_votes  tmdb_popularity  tmdb_score  
1704         NaN              NaN         NaN  


这行啥也没有，直接删了吧

In [9]:
cleaned_titles = cleaned_titles.dropna(subset=['title'])

In [10]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5849 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5849 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5849 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5849 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5849 non-null   int64  
 7   genres                5849 non-null   object 
 8   production_countries  5849 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5446 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
dtypes: float64(5), int64(2), o

把 type 为 'MOVIE' 的 NaN seasons 填成 0

In [11]:
cleaned_titles['seasons'] = cleaned_titles['seasons'].fillna(0)

转成 int，避免 float 类型（因为原本有 NaN，所以是 float）

In [12]:
cleaned_titles['seasons'] = cleaned_titles['seasons'].astype(int)

我们再来看看有什么问题

In [13]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5849 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5849 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5849 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5849 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5849 non-null   int64  
 7   genres                5849 non-null   object 
 8   production_countries  5849 non-null   object 
 9   seasons               5849 non-null   int64  
 10  imdb_id               5446 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
dtypes: float64(4), int64(3), o

好，现在其他的变量都可以为空值，接下来检查数据是否合理  
假设我们认为合理的电影年份`release_year`范围是 1880 ~ 2025，输出不合理的年份

In [14]:
invalid_years = cleaned_titles[
    (cleaned_titles['release_year'] < 1880) | 
    (cleaned_titles['release_year'] > 2025)
]

print(invalid_years[['title', 'release_year']])


Empty DataFrame
Columns: [title, release_year]
Index: []


没有，继续下一步，电影时常`runtime`是否合理？先看一下最小值和最大值

In [15]:
print("Runtime min:", cleaned_titles['runtime'].min())
print("Runtime max:", cleaned_titles['runtime'].max())

Runtime min: 0
Runtime max: 240


还可以，毕竟有些短片一分钟不到，就不用处理了

对于投票数这个变量：应该是整形int，而不是float  
先把无人投票的影片 NaN 转填成 0（或保留 NaN 也行）

In [16]:
cleaned_titles['imdb_votes'] = cleaned_titles['imdb_votes'].fillna(0)

转换为 int

In [17]:
cleaned_titles['imdb_votes'] = cleaned_titles['imdb_votes'].astype(int)

In [18]:
cleaned_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5849 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5849 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5849 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5849 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5849 non-null   int64  
 7   genres                5849 non-null   object 
 8   production_countries  5849 non-null   object 
 9   seasons               5849 non-null   int64  
 10  imdb_id               5446 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5849 non-null   int64  
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
dtypes: float64(3), int64(4), o

接下来把genres变量进行拆分，让每个类型独占一行

虽然`genres`表示形式是列表，但其实际类型并非字符串列表，而是单一个长字符串`"['drama', 'crime']"`，无法直接用`value_counts`统计各个值出现的次数。
我们可以使用Python内置的`eval`函数，它可以把字符串转换成表达式，所以可以帮我们把表示列表的字符串转换成列表本身。

eval(s) 会把这个字符串当作 Python 表达式 来执行，结果变成真正的 list：

"['drama', 'crime']" → ['drama', 'crime']***（list 类型）***。

.apply(lambda s: ...) 逐行应用，所以整列都从字符串变成了列表。

In [19]:
cleaned_titles['genres'] = cleaned_titles['genres'].apply(lambda s: eval(s))
cleaned_titles['genres'][1]

['drama', 'crime']

`explode` 方法会把一行里的列表拆开成多行，每个元素单独占一行。

In [20]:
cleaned_titles = cleaned_titles.explode("genres")

In [21]:
cleaned_titles['genres']

0       documentation
1               drama
1               crime
2               drama
2              action
            ...      
5847           comedy
5848              NaN
5849           family
5849        animation
5849           comedy
Name: genres, Length: 15146, dtype: object

总行数从5000多增到了15000多
看5行，看看它被分成了什么样子？

In [22]:
cleaned_titles.head(5)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,['US'],1,NaN,NaN,0,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],0,tt0075314,8.2,808582,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,['US'],0,tt0075314,8.2,808582,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,['US'],0,tt0068473,7.7,107673,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,['US'],0,tt0068473,7.7,107673,10.010,7.300


好，已经可以了，下面对`production_countries`做同样的操作

In [23]:
cleaned_titles['production_countries'] = cleaned_titles['production_countries'].apply(lambda s: eval(s))
cleaned_titles['production_countries'][0]

['US']

In [24]:
cleaned_titles = cleaned_titles.explode("production_countries")

In [25]:
cleaned_titles['production_countries']

0        US
1        US
1        US
2        US
2        US
       ... 
5847     CO
5848     US
5849    NaN
5849    NaN
5849    NaN
Name: production_countries, Length: 17817, dtype: object

变成了1万7千多行

In [26]:
cleaned_titles.sample(5)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1332,ts77654,Care Bears and Cousins,SHOW,Care Bears and Cousins,2015,TV-Y,22,animation,US,2,tt4680360,6.8,83,3.153,9.2
1289,tm142881,Jem and the Holograms,MOVIE,As a small-town girl catapults from undergroun...,2015,PG,118,family,US,0,tt3614530,4.1,7867,9.666,5.5
5364,tm1047190,The Phantom,MOVIE,Carlos DeLuna was arrested in 1993 aged 21 for...,2021,NaN,81,crime,GB,0,tt14597294,6.5,759,3.462,6.1
1601,tm201855,Katti Batti,MOVIE,A man falls in love with a woman because of he...,2015,NaN,138,drama,IN,0,tt4467262,4.6,3157,1.508,5.6
4265,ts218367,Secreto bien guardado,SHOW,"In the summer of 1940, Amalia, a young high-cl...",2019,TV-MA,30,romance,AR,1,tt10702412,5.0,195,2.832,10.0


国家也分开了

针对`cleaned_titles`，不一致数据可能存在于`genres`和`character`变量中，我们将查看是否存在多个不同值指代同一流派，以及多个不同值指代同一国家的情况。

In [27]:
cleaned_titles['genres'].value_counts()

genres
drama            3517
comedy           2538
thriller         1505
action           1394
romance          1098
crime            1093
documentation    1085
animation         816
family            803
fantasy           738
european          699
scifi             676
horror            451
history           336
music             289
reality           241
war               232
sport             188
western            56
Name: count, dtype: int64

不存在重复问题  
genres还需要删除无意义的空值

In [28]:
cleaned_titles = cleaned_titles.query('genres != ""')
cleaned_titles.query('genres == ""')

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


下面开始处理另一个表格`credits.csv`演员导演表

In [29]:
cleaned_credits.head(10)

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR
5,32267,tm84618,Peter Boyle,Wizard,ACTOR
6,519612,tm84618,Leonard Harris,Senator Charles Palantine,ACTOR
7,29068,tm84618,Diahnne Abbott,Concession Girl,ACTOR
8,519613,tm84618,Gino Ardito,Policeman at Rally,ACTOR
9,3308,tm84618,Martin Scorsese,Passenger Watching Silhouette,ACTOR


## **数据干净度**

In [30]:
cleaned_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


77801行，character有缺失值，但不重要，因为可以是龙套角色没有角色名  
persin_id，演职员id不应该为int64，应该为object，转换一下

In [31]:
cleaned_credits["person_id"] = cleaned_credits["person_id"].astype("str")
cleaned_credits["person_id"]

0           3748
1          14658
2           7064
3           3739
4          48933
          ...   
77796     736339
77797     399499
77798     373198
77799     378132
77800    1950416
Name: person_id, Length: 77801, dtype: object

根据数据变量的含义以及内容来看，`cleaned_titles`里不应该存在每个变量值都相同的观察值，因此查看是否存在重复值。

In [32]:
cleaned_titles.duplicated().sum()

np.int64(0)

没有重复值，针对`original_credits`，不一致数据可能存在于`role`中，我们将查看是否存在多个不同值指代同一演职员类型的情况。

In [33]:
original_credits['role'].value_counts()

role
ACTOR       73251
DIRECTOR     4550
Name: count, dtype: int64

从以上输出结果来看，`role`只有两种可能的值，`ACTOR`或`DIRECTOR`，不存在不一致数据。我们可以把这列的类型转换为`Category`，好处是比字符串类型更节约内存空间，也能表明说值的类型有限。

In [34]:
cleaned_credits["role"] = cleaned_credits["role"].astype("category")
cleaned_credits["role"]

0           ACTOR
1           ACTOR
2           ACTOR
3           ACTOR
4           ACTOR
           ...   
77796       ACTOR
77797       ACTOR
77798       ACTOR
77799       ACTOR
77800    DIRECTOR
Name: role, Length: 77801, dtype: category
Categories (2, object): ['ACTOR', 'DIRECTOR']

## **整理数据**

现在两个表都清理完了，开始整理数据  
我们的目的是把两张表**合成一张**（根据作品id）

In [35]:
cleaned_titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,US,1,NaN,NaN,0,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,0,tt0075314,8.2,808582,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,US,0,tt0075314,8.2,808582,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,US,0,tt0068473,7.7,107673,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,US,0,tt0068473,7.7,107673,10.010,7.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021,NaN,90,comedy,CO,0,tt14585902,3.8,68,26.005,6.300
5848,tm1035612,Dad Stop Embarrassing Me - The Afterparty,MOVIE,"Jamie Foxx, David Alan Grier and more from the...",2021,PG-13,37,NaN,US,0,NaN,NaN,0,1.296,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021,NaN,7,family,NaN,1,tt13711094,7.8,18,2.289,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021,NaN,7,animation,NaN,1,tt13711094,7.8,18,2.289,10.000


In [36]:
cleaned_credits

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR
...,...,...,...,...,...
77796,736339,tm1059008,Adelaida Buscato,María Paz,ACTOR
77797,399499,tm1059008,Luz Stella Luengas,Karen Bayona,ACTOR
77798,373198,tm1059008,Inés Prieto,Fanny,ACTOR
77799,378132,tm1059008,Isabel Gaona,Cacica,ACTOR


此次数据分析目标是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

那为了能同时获得流派与演员数据，我们需要把`cleaned_titles`和`cleaned_credits`，通过`id`作为键进行连接，因为两个数据表中`id`都是影视作品ID。

In [37]:
credits_with_titles = pd.merge(cleaned_credits, cleaned_titles, on="id", how="inner")

内连接（inner join） 的意思是：

只保留两张表里 都出现过的 id。

如果某个 id 只在一边表中存在，就不会出现在结果里。

In [38]:
credits_with_titles.head()

,person_id,id,name,character,role,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,0,tt0075314,8.2,808582,40.965,8.179
1,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,US,0,tt0075314,8.2,808582,40.965,8.179
2,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,0,tt0075314,8.2,808582,40.965,8.179
3,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,US,0,tt0075314,8.2,808582,40.965,8.179
4,7064,tm84618,Albert Brooks,Tom,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,0,tt0075314,8.2,808582,40.965,8.179


这是链接后的表格，由于我们只对挖掘演员的参演作品口碑感兴趣，导演不在我们的分析范围内，因此根据`role`，筛选出类型为`ACTOR`的观察值，供后续分析。

In [39]:
actor_with_titles = credits_with_titles.query('role == "ACTOR"')

为了挖掘出各个流派中的高IMDB评分作品演员，我们需要先根据流派和演员进行分组。

对演员进行分组的时候，选择的是用`person_id`而不是`name`变量，原因是名字容易出现错拼或者重名的情况，演职员ID会比演员姓名更加准确地反映是哪位演员。

In [40]:
groupby_id = actor_with_titles.groupby(["genres", "person_id"])

`.groupby(["genres", "person_id"])`这个格式.groupby方法会按照指定的列进行分组。  
第一层分组：genres → 先按照电影类型分开  
第二层分组：person_id → 在每个类型下面，再按照演员 ID 分组

分组后，我们只需要对`imdb_score`的值进行聚合计算，因此只提取`imdb_score`变量，然后调用`mean`，来计算各个流派影视作品中，每位演员参演作品的平均IMDB评分。

In [42]:
groupby_id = groupby_id["imdb_score"].mean()
groupby_id

genres   person_id
action   1000         6.866667
         100007       7.000000
         100013       6.400000
         100019       6.500000
         100020       6.500000
                        ...   
western  993735       6.500000
         998673       7.300000
         998674       7.300000
         998675       7.300000
         99940        4.000000
Name: imdb_score, Length: 172860, dtype: float64

我们可以调用`reset_index`，对层次化索引进行重置，得到更加规整的DataFrame。

In [43]:
imdb_score_groupby_genres_and_person_id_df = groupby_id.reset_index()
imdb_score_groupby_genres_and_person_id_df

,genres,person_id,imdb_score
0,action,1000,6.866667
1,action,100007,7.000000
2,action,100013,6.400000
3,action,100019,6.500000
4,action,100020,6.500000
...,...,...,...
172855,western,993735,6.500000
172856,western,998673,7.300000
172857,western,998674,7.300000
172858,western,998675,7.300000


现在针对流派和演员分组的IMDB评分数据已经整理好，可以进入后续的分析步骤了。

但我们当前可以继续做一些数据整理，比如对上面的结果再次进行分组，找出各个流派里演员作品最高的平均评分是多少、最高评分对应的演员名字是什么。

要得到这一结果，我们需要再次用`genres`进行分组，然后提取出`imdb_score`变量，计算其最大值。

In [44]:
genres_max_scores = imdb_score_groupby_genres_and_person_id_df.groupby("genres")["imdb_score"].max()
genres_max_scores

genres
action           9.3
animation        9.3
comedy           9.2
crime            9.5
documentation    9.1
drama            9.5
european         8.9
family           9.3
fantasy          9.3
history          9.1
horror           9.0
music            8.8
reality          8.9
romance          9.2
scifi            9.3
sport            9.1
thriller         9.5
war              8.8
western          8.9
Name: imdb_score, dtype: float64

在我们知道最高分后，可以把以上结果和之前得到的`imdb_score_groupby_genres_and_person_id_df`再次进行连接，得到最高分对应的各个演员ID是什么，也就是这个最高平均分是哪位演员拿到的。

In [45]:
genres_max_score_with_person_id = pd.merge(imdb_score_groupby_genres_and_person_id_df, genres_max_scores, on=["genres", "imdb_score"])
genres_max_score_with_person_id

,genres,person_id,imdb_score
0,action,12790,9.3
1,action,1303,9.3
2,action,21033,9.3
3,action,336830,9.3
4,action,86591,9.3
...,...,...,...
131,war,826547,8.8
132,western,22311,8.9
133,western,28166,8.9
134,western,28180,8.9


从以上结果可以看出，最高分对应的演员不一定只有一位，可能有多位演员的平均得分相同。

为了得到演员ID所对应的演员名字，我们可以和`cleaned_credits`这个DataFrame进行连接。这个DataFrame还有其它列，我们只需要得到`person_id`和`name`的对应，所以可以先提取出那两列，并把重复行删除。

In [46]:
actor_id_with_names = cleaned_credits[['person_id', 'name']].drop_duplicates()
actor_id_with_names.head(10)

,person_id,name
0,3748,Robert De Niro
1,14658,Jodie Foster
2,7064,Albert Brooks
3,3739,Harvey Keitel
4,48933,Cybill Shepherd
5,32267,Peter Boyle
6,519612,Leonard Harris
7,29068,Diahnne Abbott
8,519613,Gino Ardito
9,3308,Martin Scorsese


下一步就可以把`actor_id_with_names`与前面得到的`genres_max_score_with_person_id`进行连接，增加`name`变量，从而展示平均评分最高的演员名字。

In [47]:
genres_max_score_with_actor_name = pd.merge(genres_max_score_with_person_id, actor_id_with_names, on="person_id")
genres_max_score_with_actor_name

,genres,person_id,imdb_score,name
0,action,12790,9.3,Olivia Hack
1,action,1303,9.3,Jessie Flower
2,action,21033,9.3,Zach Tyler
3,action,336830,9.3,André Sogliuzzo
4,action,86591,9.3,Cricket Leigh
...,...,...,...,...
131,war,826547,8.8,Yuto Uemura
132,western,22311,8.9,Koichi Yamadera
133,western,28166,8.9,Megumi Hayashibara
134,western,28180,8.9,Unsho Ishizuka


为了把相同流派都排序在一起，我们还可以用`sort_values`方法，把结果里面的行根据`genres`进行排序，然后用`reset_index`把索引重新排序。

索引重新排序后，DataFrame会多出`index`一列，我们可以再把`index`列进行删除。

In [48]:
genres_max_score_with_actor_name = genres_max_score_with_actor_name.sort_values("genres").reset_index().drop("index", axis=1)
genres_max_score_with_actor_name

,genres,person_id,imdb_score,name
0,action,12790,9.3,Olivia Hack
1,action,1303,9.3,Jessie Flower
2,action,21033,9.3,Zach Tyler
3,action,336830,9.3,André Sogliuzzo
4,action,86591,9.3,Cricket Leigh
...,...,...,...,...
131,war,826547,8.8,Yuto Uemura
132,western,28180,8.9,Unsho Ishizuka
133,western,22311,8.9,Koichi Yamadera
134,western,28166,8.9,Megumi Hayashibara
